# Bab 2: End-to-End Machine Learning Project

Bab ini akan memandu Anda melalui sebuah proyek pembelajaran mesin secara menyeluruh, dari tahap awal hingga akhir, menggunakan dataset **California Housing Prices**. Tujuan proyeknya adalah membangun model yang dapat memprediksi harga median rumah di sebuah distrik California.

Langkah-langkah yang akan kita jalani:
- **Melihat gambaran besar** proyek dan tujuan bisnis.
- **Mengambil data** yang diperlukan untuk pelatihan.
- **Menemukan dan memvisualisasikan data** untuk mendapatkan wawasan.
- **Mempersiapkan data** untuk algoritma pembelajaran mesin.
- **Memilih model** yang sesuai dan melatihnya.
- **Menyempurnakan model** untuk meningkatkan kinerjanya.
- **Menyajikan solusi** dan mengevaluasi sistem pada data uji.
- **Meluncurkan, memantau, dan memelihara sistem** pembelajaran mesin.

---

### 1. Mendapatkan Data

Langkah pertama adalah mengunduh dan memuat dataset. Kita akan membuat beberapa fungsi untuk mengotomatiskan proses ini agar rapi dan dapat digunakan kembali.

```python
# Menyiapkan environment
import sys
assert sys.version_info >= (3, 5)

import sklearn
assert sklearn.__version__ >= "0.20"

import numpy as np
import pandas as pd
import os
import tarfile
import urllib.request
import matplotlib.pyplot as plt

# Lokasi download dan path penyimpanan data
DOWNLOAD_ROOT = "[https://raw.githubusercontent.com/ageron/handson-ml2/master/](https://raw.githubusercontent.com/ageron/handson-ml2/master/)"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

# Fungsi untuk mengunduh dan mengekstrak data
def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    os.makedirs(housing_path, exist_ok=True)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

# Menjalankan fungsi download
fetch_housing_data()

# Fungsi untuk memuat data CSV ke dalam pandas DataFrame
def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

housing = load_housing_data()
```

---

### 2. Menemukan dan Memvisualisasikan Data untuk Mendapatkan Wawasan

Setelah data dimuat, kita perlu melihat struktur, ringkasan statistik, dan visualisasinya untuk mendapatkan pemahaman awal.

``` python
# Menampilkan 5 baris pertama
print("Lima baris pertama:")
print(housing.head())
print("\n" + "="*50 + "\n")

# Menampilkan informasi ringkas (tipe data, nilai non-null)
print("Informasi dataset:")
housing.info()
print("\n" + "="*50 + "\n")

# Melihat ringkasan statistik fitur numerik
print("Deskripsi statistik:")
print(housing.describe())
print("\n" + "="*50 + "\n")

# Melihat distribusi data dengan histogram
print("Distribusi data (Histogram):")
housing.hist(bins=50, figsize=(20,15))
plt.show()

# Visualisasi geografis data
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.1,
             s=housing["population"]/100, label="population", figsize=(10,7),
             c="median_house_value", cmap=plt.get_cmap("jet"), colorbar=True,
             sharex=False)
plt.legend()
plt.show()
```

---

### 3. Membuat Test Set (Stratified Sampling)
Penting untuk menyisihkan test set sejak awal untuk mencegah data snooping bias. Kita akan menggunakan stratified sampling berdasarkan kategori pendapatan (median_income) untuk memastikan test set kita representatif terhadap keseluruhan data.

``` python
# Membuat kategori pendapatan untuk stratified sampling
housing["income_cat"] = pd.cut(housing["median_income"],
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5])

from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

# Menghapus kolom income_cat karena hanya digunakan untuk splitting
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

# Membuat salinan data training untuk eksplorasi dan persiapan
housing = strat_train_set.copy()
```

---

### 4. Mempersiapkan Data (Preprocessing Pipeline)
Tahap ini krusial, di mana kita membersihkan dan mentransformasi data agar siap digunakan oleh model. Kita akan membangun sebuah pipeline untuk menangani fitur numerik dan kategorikal secara terpisah dan terstruktur.
``` python
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

# Memisahkan fitur dari label pada data training
housing_labels = housing["median_house_value"].copy()
housing_features = housing.drop("median_house_value", axis=1)

# Mengambil nama kolom numerik dan kategorikal
housing_num_attribs = housing_features.drop("ocean_proximity", axis=1)
num_attribs = list(housing_num_attribs)
cat_attribs = ["ocean_proximity"]

# Pipeline untuk fitur numerik: mengisi nilai kosong dengan median & menstandardisasi skala
num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler()),
    ])

# Pipeline penuh yang menggabungkan proses untuk fitur numerik dan kategorikal
full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
    ])

# Menjalankan pipeline pada data training
housing_prepared = full_pipeline.fit_transform(housing_features)
```

---

### 5. Memilih dan Melatih Model
Kita akan melatih beberapa model dan mengevaluasinya menggunakan cross-validation untuk memilih kandidat model terbaik. Di sini kita contohkan dengan RandomForestRegressor.
```python
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

# Inisialisasi model
forest_reg = RandomForestRegressor(random_state=42)

# Evaluasi model menggunakan cross-validation (10-fold)
# Kita gunakan scoring 'neg_mean_squared_error', lalu akarkan untuk mendapatkan RMSE
scores = cross_val_score(forest_reg, housing_prepared, housing_labels,
                         scoring="neg_mean_squared_error", cv=10)
rmse_scores = np.sqrt(-scores)

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

print("Hasil Evaluasi RandomForestRegressor:")
display_scores(rmse_scores)
```

---

### 6. Menyempurnakan Model (Fine-Tuning)
Setelah memilih model dengan potensi terbaik, kita mencari kombinasi hyperparameter optimal menggunakan GridSearchCV.
``` python
from sklearn.model_selection import GridSearchCV

# Definisikan grid hyperparameter yang akan diuji
param_grid = [
    # Coba kombinasi n_estimators dan max_features
    {'n_estimators': [10, 30, 50], 'max_features': [6, 8, 10]},
    # Coba kombinasi lain dengan bootstrap=False
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
  ]

forest_reg = RandomForestRegressor(random_state=42)

# Menjalankan grid search
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
grid_search.fit(housing_prepared, housing_labels)

# Menampilkan hyperparameter terbaik
print("Hyperparameter terbaik:", grid_search.best_params_)
```

---

### 7. Evaluasi Akhir pada Test Set
Ini adalah langkah terakhir dalam evaluasi. Kita menggunakan test set yang telah kita sisihkan sejak awal untuk mengukur kinerja model final yang sudah disempurnakan.
```python
from sklearn.metrics import mean_squared_error

# Model terbaik dari Grid Search
final_model = grid_search.best_estimator_

# Memisahkan fitur dan label dari test set
X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()

# Menjalankan pipeline pada test set (HANYA transform, bukan fit_transform!)
X_test_prepared = full_pipeline.transform(X_test)

# Membuat prediksi
final_predictions = final_model.predict(X_test_prepared)

# Menghitung RMSE final
final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)
print("\nRMSE final pada test set:", final_rmse)
```

---

### 8. Meluncurkan, Memantau, dan Memelihara Sistem
Setelah model dianggap memuaskan, langkah selanjutnya adalah menyimpannya untuk digunakan di lingkungan produksi. Anda perlu memantau kinerjanya secara berkala dan mungkin melatihnya kembali dengan data baru.
```python
import joblib

# Menyimpan model final dan pipeline untuk penggunaan selanjutnya
joblib.dump(final_model, 'final_model.pkl')
joblib.dump(full_pipeline, 'full_pipeline.pkl')

# Contoh cara memuat kembali dan menggunakan model
# model_loaded = joblib.load('final_model.pkl')
# pipeline_loaded = joblib.load('full_pipeline.pkl')
# new_data_prepared = pipeline_loaded.transform(new_data)
# new_predictions = model_loaded.predict(new_data_prepared)
```